<a href="https://colab.research.google.com/github/Migaalee/Genomics_colab_HMI/blob/main/Parse_Xml_files_from_KEGG_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
# For general data science and matrix manipulation
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy import stats
import csv

# For pathway analysis in python
#import sspa
#import gseapy

# For plotting
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# For network creation and visualisation
import networkx as nx

#For regex
import re
import random

from google.colab import files
import zipfile

import shutil

import os
import sys
sys.path


import xml.etree.ElementTree as ET #Parse and read XML data
import tarfile #read from tarfile instead of extracting all data
import xml.etree.ElementTree as ET
from collections import Counter

import pprint as pp
import pickle
import site
site.getsitepackages()
from collections import defaultdict #I'm importing this library so we have a code with less lists!!

In [15]:
uploaded = files.upload()

Saving KEGG.zip to KEGG (2).zip


In [17]:
#zip_file_name = list(uploaded.keys())[0]
zip_file_name = next(iter(uploaded))


In [18]:
extract_to = 'unzipped_files'
if not os.path.exists(extract_to):
    os.makedirs(extract_to)

In [19]:
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

In [20]:
csv_path = 'csv_files'
if not os.path.exists(csv_path):
    os.makedirs(csv_path)

In [ ]:
tree = ET.parse('dme01100.xml')
root = tree.getroot()

In [41]:
for xml_file in os.listdir(extract_to):
    if xml_file.endswith('.xml'):
        # Parse the XML file
        tree = ET.parse(os.path.join(extract_to, xml_file))
        root = tree.getroot()

        # Extract specific data
        cpd_entries = []
        for entry in root.findall('entry'):
            name = entry.attrib.get('name', '')
            if name.startswith('cpd:'):
                entry_details = {
                    'id': entry.attrib.get('id'),
                    'name': name,
                    'type': entry.attrib.get('type'),
                    'link': entry.attrib.get('link'),
                }
                cpd_entries.append(entry_details)

        # Define the CSV file name based on the XML file name
        csv_file_name = os.path.splitext(xml_file)[0] + '.csv'
        # The path where the CSV file will be saved
        csv_path = os.path.join(extract_to, csv_file_name)

        # Export the data to a CSV file
        with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
            fieldnames = ['id', 'name', 'type', 'link']
            writer = csv.DictWriter(csv_file, fieldnames=fieldnames, delimiter=';')
            writer.writeheader()
            for entry in cpd_entries:
                writer.writerow(entry)

        print(f"Processed {xml_file} to {csv_file_name}")

Processed Galactose metabolism.xml to Galactose metabolism.csv
Processed Glycosphingolipid biosynthesis.xml to Glycosphingolipid biosynthesis.csv
Processed Sulfur metabolism.xml to Sulfur metabolism.csv
Processed Inositol phosphate metabolism.xml to Inositol phosphate metabolism.csv
Processed Mucin type O-glycan biosynthesis.xml to Mucin type O-glycan biosynthesis.csv
Processed Glycine, serine and threonine metabolism.xml to Glycine, serine and threonine metabolism.csv
Processed Ether lipid metabolism.xml to Ether lipid metabolism.csv
Processed Tryptophan metabolism.xml to Tryptophan metabolism.csv
Processed Phenylalanine, tyrosine and tryptophan biosynthesis.xml to Phenylalanine, tyrosine and tryptophan biosynthesis.csv
Processed Cysteine and methionine metabolism.xml to Cysteine and methionine metabolism.csv
Processed Selenocompound metabolism.xml to Selenocompound metabolism.csv
Processed Drug metabolism - cytochrome P450.xml to Drug metabolism - cytochrome P450.csv
Processed Caffei

Only compound names extract

In [46]:
for xml_file in os.listdir(extract_to):
    if xml_file.endswith('.xml'):
        # Parse the XML file
        tree = ET.parse(os.path.join(extract_to, xml_file))
        root = tree.getroot()

        # Extract specific data
        cpd_names = []  # We'll only store the modified names here
        for entry in root.findall('.//entry'):  # Ensure we search all 'entry' elements, regardless of nesting
            name = entry.attrib.get('name', '')
            if name.startswith('cpd:'):
                modified_name = name.split("cpd:")[1]  # Keep only the part after 'cpd:'
                cpd_names.append([modified_name])  # Append as a list to match the writerows format later

        # Define the CSV file name based on the XML file name
        csv_file_name = os.path.splitext(xml_file)[0] + '.csv'
        # The path where the CSV file will be saved
        csv_path = os.path.join(extract_to, csv_file_name)

        # Export the data to a CSV file
        with open(csv_path, 'w', newline='', encoding='utf-8') as csv_file:
            # The column header will be the XML file name without the extension
            writer = csv.writer(csv_file, delimiter=';')
            writer.writerow([os.path.splitext(xml_file)[0]])  # Write the header as the file name
            writer.writerows(cpd_names)  # Write the rows of modified names

        print(f"Processed {xml_file} to {csv_file_name}")

Processed Galactose metabolism.xml to Galactose metabolism.csv
Processed Glycosphingolipid biosynthesis.xml to Glycosphingolipid biosynthesis.csv
Processed Sulfur metabolism.xml to Sulfur metabolism.csv
Processed Inositol phosphate metabolism.xml to Inositol phosphate metabolism.csv
Processed Mucin type O-glycan biosynthesis.xml to Mucin type O-glycan biosynthesis.csv
Processed Glycine, serine and threonine metabolism.xml to Glycine, serine and threonine metabolism.csv
Processed Ether lipid metabolism.xml to Ether lipid metabolism.csv
Processed Tryptophan metabolism.xml to Tryptophan metabolism.csv
Processed Phenylalanine, tyrosine and tryptophan biosynthesis.xml to Phenylalanine, tyrosine and tryptophan biosynthesis.csv
Processed Cysteine and methionine metabolism.xml to Cysteine and methionine metabolism.csv
Processed Selenocompound metabolism.xml to Selenocompound metabolism.csv
Processed Drug metabolism - cytochrome P450.xml to Drug metabolism - cytochrome P450.csv
Processed Caffei

In [47]:
directory_path = '/content/unzipped_files'


In [48]:
zipfile_name = '/content/unzipped_files.zip'


compress folder to download

In [49]:
shutil.make_archive(zipfile_name[:-4], 'zip', directory_path)


'/content/unzipped_files.zip'

In [50]:
files.download(zipfile_name)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
/content/unzipped_files

In [27]:
extract_to = 'unzipped_files/KEGG'  # The directory where your XML files are located
xml_file_name = '2-Oxocarboxylic acid metabolism.xml'  # The name of the XML file you want to process
xml_file_path = os.path.join(extract_to, xml_file_name)  # Construct the full path


In [28]:
extract = process_xml_file(xml_file_path)


In [ ]:
#for elem in root:
    #print(f"ID: {elem.attrib['id']}, Name: {elem.attrib['name']}, Type: {elem.attrib['type']}")

In [ ]:
#for elem in root:
    #print(elem.tag, elem.attrib)

In [ ]:
cpd_entries = []
for entry in root.findall('entry'):
    name = entry.attrib.get('name', '')  # get name attribute
    if name.startswith('cpd:'):  # if name starts with "cpd:"
        entry_details = {
            'id': entry.attrib['id'],
            'name': name,
            'type': entry.attrib['type'],
            'link': entry.attrib['link'],
        }
        cpd_entries.append(entry_details)

In [ ]:
#cpd_entries

In [ ]:
df = pd.DataFrame(cpd_entries)


In [ ]:
df

,id,name,type,link
0,7894,cpd:C22834,compound,https://www.kegg.jp/dbget-bin/www_bget?C22834
1,7888,cpd:C03852,compound,https://www.kegg.jp/dbget-bin/www_bget?C03852
2,7887,cpd:C18041,compound,https://www.kegg.jp/dbget-bin/www_bget?C18041
3,7886,cpd:C18042,compound,https://www.kegg.jp/dbget-bin/www_bget?C18042
4,7885,cpd:C22832,compound,https://www.kegg.jp/dbget-bin/www_bget?C22832
...,...,...,...,...
3144,7889,cpd:C03917,compound,https://www.kegg.jp/dbget-bin/www_bget?C03917
3145,3643,cpd:C00606,compound,https://www.kegg.jp/dbget-bin/www_bget?C00606
3146,4072,cpd:C05527,compound,https://www.kegg.jp/dbget-bin/www_bget?C05527
3147,4323,cpd:C00053,compound,https://www.kegg.jp/dbget-bin/www_bget?C00053


No charts were generated by quickchart


In [ ]:
df.to_csv("dme01100.csv", sep=";")

In [ ]:
files.download("dme01100.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>